# Example 2: Extract method

[Extract method in the refactoring catalog.](http://refactoring.com/catalog/extractMethod.html)

---

Say you're keeping track of the grades for a set of students in a class. This code works, but the `print_stats` function has a lot going on. If you want to compute any more stats (like median) it's going to get ugly.

In [3]:
class Grade:
    def __init__(self, student, score):
        self.student = student
        self.score =  score


def print_stats(grades):
    if not grades:
        raise ValueError('Must supply at least one Grade')
        
    total, count = 0, 0
    low, high = float('inf'), float('-inf')
    for grade in grades:
        total += grade.score
        count += 1
        if grade.score < low:
            low = grade.score
        elif grade.score > high:
            high = grade.score

    average = total / count

    print('Average score: %.1f, low score: %.1f, high score %.1f' %
          (average, low, high))

In [4]:
grades = [Grade('Bob', 92), Grade('Sally', 89), Grade('Roger', 73), Grade('Alice', 96)]
print_stats(grades)

Average score: 87.5, low score: 73.0, high score 96.0


---

One common way people try to make this more readable is with a closure because it at least isolates the inside of the loop. This is annoying because you need to use the `nonlocal` keyword to make sure the closure works. In Python 2 it's even worse because `nonlocal` isn't available.

In [5]:
def print_stats(grades):
    if not grades:
        raise ValueError('Must supply at least one Grade')
        
    total, count = 0, 0
    low, high = float('inf'), float('-inf')

    def adjust_stats(grade):
        nonlocal total, count, low, high
        total += grade.score
        count += 1
        if grade.score < low:
            low = grade.score
        elif grade.score > high:
            high = grade.score

    for grade in grades:
        adjust_stats(grade)
            
    average = total / count

    print('Average score: %.1f, low score: %.1f, high score %.1f' %
          (average, low, high))

In [6]:
print_stats(grades)

Average score: 87.5, low score: 73.0, high score 96.0


---

What's better is to split the inner closure into a helper class. You make the helper class having a single entrypoint named `__call__` so it acts like a plain function. The presence of `__call__` is a hint to the reader than the purpose of the class is to be a stateful closure.

In [7]:
class CalculateStats:
    def __init__(self):
        self.total = 0
        self.count = 0
        self.low = float('inf')
        self.high = float('-inf')

    def __call__(self, grades):
        for grade in grades:
            self.total += grade.score
            self.count += 1
            if grade.score < self.low:
                self.low = grade.score
            elif grade.score > self.high:
                self.high = grade.score

                
def print_stats(grades):
    if not grades:
        raise ValueError('Must supply at least one Grade')

    stats = CalculateStats()
    stats(grades)
    average = stats.total / stats.count

    print('Average score: %.1f, low score: %.1f, high score %.1f' %
          (average, stats.low, stats.high))

In [8]:
print_stats(grades)

Average score: 87.5, low score: 73.0, high score 96.0


---

You can even add other properties to this closure to give the illusion it's doing more bookkeeping than it really is.

In [9]:
class CalculateStats:
    def __init__(self):
        self.total = 0
        self.count = 0
        self.low = float('inf')
        self.high = float('-inf')

    def __call__(self, grades):
        for grade in grades:
            self.total += grade.score
            self.count += 1
            if grade.score < self.low:
                self.low = grade.score
            elif grade.score > self.high:
                self.high = grade.score

    @property
    def average(self):
        return self.total / self.count

    
def print_stats(grades):
    if not grades:
        raise ValueError('Must supply at least one Grade')

    stats = CalculateStats()
    stats(grades)

    print('Average score: %.1f, low score: %.1f, high score %.1f' %
          (stats.average, stats.low, stats.high))

In [10]:
print_stats(grades)

Average score: 87.5, low score: 73.0, high score 96.0


---

If you need more than one entrypoint method, you probably need to redraw the boundaries of responsibility between the classes and go for real method names, not just `__call__`.